In [49]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.csg import *
import numpy as np
import matplotlib.pyplot as plt

## POINT 1

In [50]:
outer = Rectangle(1,1).Face()
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"

mesh = Mesh(OCCGeometry(outer, dim=2).GenerateMesh(maxh=0.05))
fes = H1(mesh, order=2, dirichlet="r|l|b|t")

b = GridFunction(VectorH1(mesh))
b.Set((10,2), definedon="*")


u, v = fes.TnT()
gfu = GridFunction(fes)
# gfu.Set(mesh.BoundaryCF({'l':0}), definedon=mesh.Boundaries("l"))


a = BilinearForm(fes)
a += (grad(u)*grad(v)-InnerProduct(b, u*grad(v)))*dx #-InnerProduct(b, u*grad(v))
a.Assemble()

f = LinearForm(fes)
f+=v*dx
f.Assemble()

r = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r




Draw(gfu, mesh, vectors=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## POINT 2
### Strong form

$-\Delta u + \nabla(\vec{b}u) = 1$


If we hit both hand sides with $\int\limits_{\Omega} \star vdV$
and perform integration by parts. Since operations are true from both directions (e.g $\Leftrightarrow$) we prove the estrong form.

$\int\limits_{\Omega}\Delta u \cdot v dV = \int\limits_{\Gamma} \partial_n u \cdot v dS - \int\limits_{\Omega} \nabla u \cdot \nabla v dV$

$\int\limits_{\Omega}\nabla(bu) \cdot v dV = \int\limits_{\Gamma} bu \cdot n \cdot v dS - \int\limits_{\Omega} bu \cdot \nabla v dV$


$\int\limits_{\Omega}1 v dV =\int\limits_{\Omega}1 v dV$

Interpretation: Diffusion, transport and reaction combined. Very commonly found on dynamical systems such as cell processes.
Although, usually we deal with non equilibrium cases


## POINT 3

CPU: Apple  M1


Run time 1.7 sec 

In [52]:
outer = Rectangle(1,1).Face()
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"

mesh = Mesh(OCCGeometry(outer, dim=2).GenerateMesh(maxh=0.01))
fes = H1(mesh, order=2, dirichlet="r|l|b|t")

b = GridFunction(VectorH1(mesh))
b.Set((100,100), definedon="*")


u, v = fes.TnT()
gfu = GridFunction(fes)
# gfu.Set(mesh.BoundaryCF({'l':0}), definedon=mesh.Boundaries("l"))


a = BilinearForm(fes)
a += (grad(u)*grad(v)-InnerProduct(b, u*grad(v)))*dx #-InnerProduct(b, u*grad(v))
a.Assemble()

f = LinearForm(fes)
f+=v*dx
f.Assemble()

r = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r




Draw(gfu, mesh, vectors=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## POINT 4

In this scenario the transport part is dominant and affects the direction of the gradient of the solution dratically. 
Also since the amplitude of the vector is larger we have to use better resolution.

We start with the billinear form and apply integration by parts at the convection term.

$A(u,u) = \int\limits_{\Omega}\left[(\nabla u)^2  - b u\nabla u \right] dV$

$A(u,u) = \int\limits_{\Omega}\left[(\nabla u)^2  - b \nabla u^2 \right] dV$

$= \int\limits_{\Omega}\left[(\nabla u)^2\right] dV - \underbrace{\int\limits_{\Gamma} b u^2 dS}_{=0} + \underbrace {\int\limits_{\Omega} \nabla b \cdot u^2 dV}_{=0}  $

$ =\int\limits_{\Omega}\left[(\nabla u)^2\right] dV = ||u||_V $

$\geq ||u||_V$